In [ ]:
# Using weather with current best model (v2)

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression, LogisticRegression
import pickle
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm


In [2]:
SQLPW=''

In [3]:
# Connect and query
engine = create_engine("mysql+mysqlconnector://softies:" + SQLPW + "@db-bikes.ck7tnbvjxsza.eu-west-1.rds.amazonaws.com:3306/db-bikes")

# Test with just station 108
stations = pd.read_sql_query("SELECT dynamic.available_bikes, dynamic.last_update from dynamic JOIN static ON static.address=dynamic.address WHERE static.number=108", engine)

In [6]:
weather = pd.read_sql_query("SELECT weather.date, weather.temperature, weather.rainfall from weather", engine)

# rename so both dfs have same name on time column
weather.rename(columns={'date':'last_update'},inplace=True)

In [7]:
weather

,last_update,temperature,rainfall
0,2020-01-01 09:00:00,7,0
1,2022-02-20 18:00:00,4,0.0
2,2022-02-22 12:00:00,9,0.1
3,2022-02-22 13:00:00,9,0.0
4,2022-02-22 15:00:00,10,0.0
...,...,...,...
1113,2022-04-11 16:00:00,11,0.0
1114,2022-04-11 17:00:00,11,0.0
1115,2022-04-11 18:00:00,12,0.0
1116,2022-04-11 19:00:00,12,0.0


In [8]:
stations

,available_bikes,last_update
0,9,2022-02-22 15:43:46
1,9,2022-02-22 15:53:52
2,18,2022-02-23 18:03:46
3,17,2022-02-23 18:17:57
4,17,2022-02-23 18:28:04
...,...,...
8245,17,2022-04-11 19:42:46
8246,17,2022-04-11 19:52:53
8247,16,2022-04-11 19:54:56
8248,16,2022-04-11 20:05:02


In [9]:
# merge both dfs on closest times
stations = pd.merge_asof(stations, weather, on="last_update", direction='nearest')

In [11]:
stations

,available_bikes,last_update,temperature,rainfall
0,9,2022-02-22 15:43:46,10,0.0
1,9,2022-02-22 15:53:52,10,0.0
2,18,2022-02-23 18:03:46,8,0.2
3,17,2022-02-23 18:17:57,8,0.2
4,17,2022-02-23 18:28:04,8,0.2
...,...,...,...,...
8245,17,2022-04-11 19:42:46,11,0.0
8246,17,2022-04-11 19:52:53,11,0.0
8247,16,2022-04-11 19:54:56,11,0.0
8248,16,2022-04-11 20:05:02,11,0.0


In [13]:
features = ['time', 'day', 'temperature', 'rainfall']
days_of_week = ["Sunday","Monday", "Tuesday","Wednesday", "Thursday", "Friday", "Saturday"]

# Split last update to day and time
stations['day'] = list(map(lambda x: x.strftime('%A'), list(stations['last_update'])))
stations['time'] = list(map(lambda x: x.strftime('%H'), list(stations['last_update'])))

# Another sexual lambda function to convert day of week to number
# stations['day'] = list(map(lambda x: days_of_week.index(x), list(stations['day'])))

# for day in days_of_week:
#     stations[day] = list(map(lambda x: 1 if x == days_of_week.index(day) else 0, list(stations['day'])))

stations

,available_bikes,last_update,temperature,rainfall,day,time
0,9,2022-02-22 15:43:46,10,0.0,Tuesday,15
1,9,2022-02-22 15:53:52,10,0.0,Tuesday,15
2,18,2022-02-23 18:03:46,8,0.2,Wednesday,18
3,17,2022-02-23 18:17:57,8,0.2,Wednesday,18
4,17,2022-02-23 18:28:04,8,0.2,Wednesday,18
...,...,...,...,...,...,...
8245,17,2022-04-11 19:42:46,11,0.0,Monday,19
8246,17,2022-04-11 19:52:53,11,0.0,Monday,19
8247,16,2022-04-11 19:54:56,11,0.0,Monday,19
8248,16,2022-04-11 20:05:02,11,0.0,Monday,20


In [14]:
X = stations[features]
X

,time,day,temperature,rainfall
0,15,Tuesday,10,0.0
1,15,Tuesday,10,0.0
2,18,Wednesday,8,0.2
3,18,Wednesday,8,0.2
4,18,Wednesday,8,0.2
...,...,...,...,...
8245,19,Monday,11,0.0
8246,19,Monday,11,0.0
8247,19,Monday,11,0.0
8248,20,Monday,11,0.0


In [15]:
# Check for nulls as a result of merging weather
X.isnull().values.any()

False

In [16]:
X.dtypes

time           object
day            object
temperature    object
rainfall       object
dtype: object

In [18]:
X['temperature'] = X['temperature'].astype('int')
X['rainfall'] = X['rainfall'].astype('float')

C:\Users\Harvey1\AppData\Local\Temp/ipykernel_9760/465827205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['temperature'] = X['temperature'].astype('int')
C:\Users\Harvey1\AppData\Local\Temp/ipykernel_9760/465827205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['rainfall'] = X['rainfall'].astype('float')


In [19]:
# Convert categorical variables into dummy variables
X = pd.get_dummies(data=X, drop_first=True)
X.shape

(8250, 31)

In [20]:
Y = stations['available_bikes']

In [21]:
# See time and day are dummy variables, temp and rainfall stay the same
X

,temperature,rainfall,time_01,time_02,time_03,time_04,time_05,time_06,time_07,time_08,...,time_20,time_21,time_22,time_23,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,10,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,8,0.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,8,0.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,8,0.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,11,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8246,11,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8247,11,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8248,11,0.0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [22]:
# Create test and train sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 40)


In [23]:
model = LinearRegression()
model.fit(X_train, Y_train)
y_test_pred = model.predict(X_test)

In [24]:
print("Mean absolute error =", round(sm.mean_absolute_error(Y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(Y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(Y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(Y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(Y_test, y_test_pred), 2))

Mean absolute error = 5.59
Mean squared error = 45.91
Median absolute error = 4.98
Explain variance score = 0.22
R2 score = 0.22
